In [4]:
from selenium import webdriver
import pandas as pd
from datetime import date
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import urllib.request
import os
import win32com.client as win32
import getpass
import chromedriver_autoinstaller



chromedriver_autoinstaller.install() 

chrome_options = webdriver.ChromeOptions()
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])


global driver


ModuleNotFoundError: No module named 'chromedriver_autoinstaller'

In [ ]:
def password_entry():
    global df_d
    driver = webdriver.Chrome()
    #driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chrome.exe",)
    driver.get("https://yil-intranet.in.ykgw.net/forms/Login.asp?Path=/OLTS/CEC/Reports/PendingList.aspx")

    print("Enter Username:")
    username_input=input()
    #username_input="220255"
    print("Enter Password:")
    user=getpass.getuser()
    password_input = getpass.getpass(user)
    print(password_input)
    #password_input="Yokogawa@feb22"
    username = driver.find_element_by_id("EmployeeNo")
    password = driver.find_element_by_id("Password")

    #username.send_keys("220255")
    #password.send_keys("Yokogawa@feb22")

    username.send_keys("",username_input)
    password.send_keys("",password_input)
    
    driver.find_element_by_name("Submit").click()
    driver.find_element_by_name("ibt1").click()
    driver.find_element_by_id("CE1_today").click()

    #today = date.today()
    Status = driver.find_element_by_id("ddlStatus")
    Status.send_keys("PendingSubmissions")

    driver.find_element_by_name("Submit").click()

    innerHTML = driver.page_source
    soup = BeautifulSoup(innerHTML, "html.parser")

    table = soup.find('table', {"id" : 'tbPendingList'})


    df_d = pd.read_html(str(table))[0]
    
          

In [ ]:
password_entry()

In [ ]:
#df2 = pd.read_excel('Extracted Data.xlsx')
global df_d

new_header = df_d.iloc[0] #grab the first row for the header
df_d = df_d[1:] #take the data less the header row
df_d.columns = new_header


In [ ]:
#driver = webdriver.Chrome(r'C:\Users\464m0359\Downloads\chromedriver_win32/chromedriver',options=chrome_options)
driver = webdriver.Chrome()
driver.get("https://yil-intranet.in.ykgw.net/Employee/Search/Search.asp")
Organisation=driver.find_element_by_name("OrganizationId")
Organisation.send_keys("All")
#driver.find_element_by_id("CE1_today").click()

number=driver.find_element_by_name("NoOfRecords")
number.send_keys("1000")

driver.find_element_by_xpath("/html/body/form/table/tbody/tr[5]/td[2]/select/option[11]").click()



In [ ]:
#today = date.today()
#Status = driver.find_element_by_id("OrgDepartmentId")
#Status.send_keys("GDC-I EXPORTS")

driver.find_element_by_name("Submit").click()

innerHTML = driver.page_source
soup = BeautifulSoup(innerHTML, "html.parser")

In [ ]:
table = soup.find_all("table")


df = pd.read_html(str(table))[1]

#driver.close()

In [ ]:
#df.to_excel(r'Muthu_Extract.xlsx',index=False,header=True)
df = df[df.Organization != "Yokogawa India Limited"]
df
df =df[:-2]
df1 = df[['Organization', 'Emp No.','Name','Office','New E-Mail']]
#df1.to_excel(r'Muthu_Extract.xlsx',index=False,header=True)



In [ ]:
df_final = pd.merge(df_d, 
                      df1, 
                      on ='Emp No.',
                      how ='right')

df_final=df_final.drop(['Organization_y', 'Name_y'], axis=1)
df_final=df_final.dropna()
df_final.to_excel("Extact.xlsx")
df_final

In [ ]:
df_mail=pd.read_excel('Extact.xlsx')
df_mail = df_mail[df_mail['New E-Mail'] != "-"]
df_mail


In [ ]:
for index, row in df_mail.iterrows():
    print("Mailing to " + row["New E-Mail"] )
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = row["New E-Mail"]
    mail.cc = "muthu.vivek@yokogawa.com"
    mail.Subject = 'This mail was automated from python please do ignore this'
    mail.Body = 'Test mail using Python '
    #mail.HTMLBody = '<h2>HTML Message body</h2>' #this field is optional

    # To attach a file to the email (optional):
    #attachment  = "D:\GRT\GRT_Overview.pptx"
    #mail.Attachments.Add(attachment)
    mail.Send()